<a href="https://colab.research.google.com/github/NadaMohammedB/Weekly-Tasks/blob/main/Copy_of_langChain_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Tutorial Exercises


In this notebook, you will practice using LangChain to interact with large language models (LLMs),
build chains, agents, and utilize memory. Fill in the code blocks with your implementations.


## Exercise 1: Basic LLM Query

In this exercise, you will set up a basic interaction with the GROQ LLaMA model using LangChain.

1. Initialize the LLM (Use GROQ and chose LLM).
2. Create a prompt that asks the LLM to generate a story about a topic.
3. Run the LLM chain to retrieve the response.

**Steps**:
- Import required modules from `langchain`.
- Initialize the LLM with your GROQ API key.
- Create a prompt template that takes a topic as input.
- Create an LLM Chain and run it to get a response.

Fill in the code below:


In [20]:
!pip install langchain langchain-community langchain-groq duckduckgo-search geopy requests

In [21]:
groq_api_key = "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"

In [22]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model = 'llama-3.1-8b-instant',
    temperature= 0,
    max_tokens= None,
    timeout = None,
    max_retries=2,
    api_key = groq_api_key
)

In [23]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=['topic'],
    template="Tell me a story about {topic}"
)

In [24]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt)

response = chain.run("happy kid")
print(response)

Once upon a time, in a small village surrounded by lush green fields and a sparkling river, there lived a happy kid named Max. Max was a bright and cheerful 8-year-old boy with a contagious smile and sparkling blue eyes. He loved nothing more than playing outside and exploring the world around him.

Max lived with his loving parents and little sister, Emma, in a cozy little house with a big backyard where he could run and play to his heart's content. Every morning, Max would wake up early, jump out of bed, and rush to the kitchen to help his mom make breakfast. He loved helping her mix batter for pancakes or scrambling eggs for breakfast.

After breakfast, Max would head out to the backyard to play with his friends. They would spend hours playing tag, hide-and-seek, or riding their bikes down the hill. Max loved the feeling of the wind in his hair and the sun on his face as he rode his bike.

But Max's favorite thing to do was help his dad in the garden. His dad was a master gardener, 

## Exercise 2: Building a Conversational Agent


In this exercise, you will create a conversational agent that can interact with a user, make decisions,
and use external tools like a search tool.

1. Define a tool.
2. Create an agent that can decide whether to use the tool or interact with the LLM.
3. Run the agent with various inputs.

**Steps**:
- Define the search tool using a function.
- Initialize an agent using the tool and the LLM.
- Run the agent with sample inputs.

Fill in the code below:


In [27]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, AgentType
from langchain_groq import ChatGroq

search = DuckDuckGoSearchRun()
tools = [search]

agent = initialize_agent(
    tools= tools,
    llm = llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)
response = agent.run("what is IoT?")
print(response)

Agent stopped due to iteration limit or time limit.


## Exercise 3: Using LLM as Memory


In this exercise, you will use an LLM to summarize and retain information from conversations.

1. Set up LLM-based memory.
2. Create a conversation with the LLM and memory.
3. Ask follow-up questions using memory to retrieve past context.

**Steps**:
- Initialize summarization-based memory.
- Run a few queries and retrieve responses.
- Ask follow-up questions that reference previous interactions.

Fill in the code below:


In [28]:
from langchain.memory import ConversationBufferMemory
from langchain import ConversationChain

memory= ConversationBufferMemory()

memory.clear()

chain= ConversationChain(llm=llm, memory=memory)

In [29]:
answr1 = chain.run("give me one letter from a to e")
answr2 = chain.run("give me one letter from f to z")
answr3 = chain.run('From the previous conversations. What were the two previuosly selected letters?')

print(f'First Response: {answr1}\nSecond Response: {answr2}\nThird Response: {answr3}')

First Response: I'd be happy to provide you with a random letter from A to E. Here's the letter "C".
Second Response: I'd be happy to provide you with a random letter from F to Z. Here's the letter "K".
Third Response: Based on our previous conversations, the two letters that were previously selected are "C" and "K".


## Exercise 4: Combining Tools and Memory


In this final exercise, you will build an intelligent agent that can use both tools (like an API) and memory.

1. Define an external tool (like a weather API).
2. Set up an agent that uses both the tool and LLM memory.
3. Interact with the agent, combining memory and external data.

**Steps**:
- Define a weather API tool (mock or real API).
- Initialize the agent with memory and the tool.
- Run the agent with inputs, and check how it uses both memory and tools.

Fill in the code below:


In [31]:
import requests
from geopy.geocoders import Nominatim

def weather(city_name: str):

    geolocator = Nominatim(user_agent="LLMexercise")

    location = geolocator.geocode(city_name)

    if location:
        latitude = location.latitude
        longitude = location.longitude
    else:
        return f"City '{city_name}' not found."

    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"

    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()

        current_weather = weather_data.get('weather', {})
        temperature = current_weather.get('temp')
        windspeed = current_weather.get('windspeed')
        winddirection = current_weather.get('winddirection')
        weather_time = current_weather.get('time')

        return (
            f"Current weather in {city_name}:\n"
            f"Temp: {temperature}°C\n"
            f"Wind Speed: {windspeed} m/s\n"
            f"Wind Direction: {winddirection}°\n"
            f"Time: {weather_time}"
        )
    else:
        return "Error."

city_name = "New York"
weather_info = weather(city_name)
print(weather_info)

Current weather in New York:
Temp: None°C
Wind Speed: None m/s
Wind Direction: None°
Time: None


In [32]:
from langchain.agents import Tool

weather = Tool(
    name="weather",
    func=weather,
    description=" Get the weather "
)

In [33]:
from langchain.agents import initialize_agent, AgentType

tools = [weather]

agent = initialize_agent(
    tools = tools,
    llm = llm,
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [35]:
response = agent.run("What is the weather in New York?")
print(response)

Agent stopped due to iteration limit or time limit.


## Done by Nada alotaibi , Lana mohammed and Amal alqahtani